In [2]:
import pandas as pd
from datetime import timedelta, time
#from datetime import datetime
import numpy as np
#import yfinance as yf
import matplotlib.pyplot as plt
#from pandas.tseries.offsets import BMonthEnd
#import pickle
import joblib
#from ta.momentum import RSIIndicator
#from ta.trend import MACD
import itertools
import xgboost as xgb
from sklearn import metrics
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix, precision_recall_curve # Loading required libraries
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

#import openinsider_scraper as OIScraper
import features as ft

In [4]:
df = joblib.load('saved_vars/35_features_2015_21-10-2025.joblib')
print('/nData loaded in from joblib file./n')
df = ft.update_latest_data(df)

df_by_ticker = df.copy()
print(df_by_ticker.shape)
#remove rows with nans or neg nums in certain rows.
df_by_ticker = df_by_ticker.dropna()
df_by_ticker = df_by_ticker[(df_by_ticker[['last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'Owned_norm', 'title_rank', 'filing_lad_days', 'same_day_trade_count']] >= 0).all(axis=1)]
print(df_by_ticker.shape)

2025-10-23 12:39:51,963 - openinsider - INFO - Starting scraping process...


/nData loaded in from joblib file./n
Scraping data


Processing months: 100%|██████████| 1/1 [00:00<00:00, 50.81it/s]
2025-10-23 12:39:51,985 - openinsider - INFO - Scraping completed. Found 286 transactions.
2025-10-23 12:39:51,989 - openinsider - INFO - Data saved to data\insider_trades.csv


Cleaning data


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:73: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[**********************82%**************         ]  117 of 142 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: VHAIO"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: LKSP"}}}
[*********************100%***********************]  142 of 142 completed

4 Failed downloads:
['PHXE.', 'DMII', 'LKSP', 'VHAIO']: YFTzMissingError('possibly delisted; no timezone found')


Adding features, this may take a while.
Starting create features


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())


Finish create features
Starting count_same_day_trades
Finished count_same_day_trades
Removeing trades with out of range or missing values.
(147703, 35)
(89430, 35)


In [5]:
todays_trades = df_by_ticker[df_by_ticker['eff_trans_date'] == df_by_ticker.iloc[-1]['eff_trans_date']]
df_by_ticker = df_by_ticker[df_by_ticker['eff_trans_date'] != df_by_ticker.iloc[-1]['eff_trans_date']]
df_by_ticker.shape

(89413, 35)

### Predicting tomorrows movements.

In [8]:
clf = xgb.XGBClassifier()
clf.load_model("saved_vars/XGBoost/23_10_2025_none_rand_spli_accuracy_0.85_threshold_0.736.json")
chosen_threshold = 0.736

In [10]:
todays_trades_dropped = todays_trades.drop([
    'transaction_date', 'trade_date', 'eff_trans_date', 
    'ticker','company_name', 'owner_name', 'prev_prices', 'future_prices', 
    'Title', 'transaction_type', 'Owned','trade_date_epoch', 'eff_trans_date_epoch',
    'last_price'
    ], axis=1)

#display(todays_trades_dropped)

todays_trades_proba = clf.predict_proba(todays_trades_dropped)[:, 1]
todays_trades_pred = (todays_trades_proba > chosen_threshold).astype(int)

print(todays_trades_proba)
print(todays_trades_pred)
display(todays_trades[todays_trades_pred==1])

[0.05090527 0.09478866 0.10581561 0.27670613 0.11243728 0.0195622
 0.01740121 0.0198191  0.01256618 0.45860058 0.6201053  0.13238823
 0.536116   0.7604988  0.6763772  0.7110436  0.7043482 ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


,transaction_date,trade_date,ticker,company_name,owner_name,Title,transaction_type,last_price,Qty,shares_held,...,macd,volume_zscore,price_vs_sma50,filing_lad_days,month_end_flag,same_day_trade_count,day_sin,day_cos,ATR_14,Vol_Ratio_20
147699,2025-10-21 19:07:08,2025-10-14,LCGMF,Lion Copper & Gold Corp.,Alford Tony L,"Dir, 10%",P - Purchase,0.1,39000.0,143014463.0,...,-0.004606,-0.261675,0.917431,7.0,0,1,0.781831,0.62349,0.007143,0.921241
